<h1>T1 Coleta, Preparação e Analise de Dados</h1>

Faça um crawler capaz de navegar por todas as páginas de países e baixar seus HTMLS.

Vitor Delela, ...

Com a biblioteca Beautiful Soup vimos como é possível tratar documentos HTML como objetos no modelo DOM, buscando informação pelas *tags* e seus atributos. Porém, para chegarmos ao ponto de processar um documento HTML precisamos realizar o download do documento a partir do seu servidor web e como descobrir novos documentos a partir de um link de origem.

Neste notebook, veremos como é possível fazer requisições HTTP utilizando a biblioteca urllib e como lidar com alguns problemas básicos que podem acontecer quando estamos trabalhando com um crawler.


Primeiro passo é importar o módulo request da biblioteca urllib. Não é necessário instalar, pois é uma biblioteca nativa do Python.

In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup



O objetivo de um crawler é descobrir, a partir de um link raiz, outros documentos que estejam linkados de alguma forma.

Uma maneira simples de realizar isto é filtrando pelas tags de links de um documento HTML. Com isto, encontramos todos os links dentro de um documento.

Abaixo, um exemplo utilizando como raiz a página do ator Kevin Bacon na Wikipedia.

Evidentemente, nem todos links serão de nosso interesse. Podemos filtrar apenas os links que nos interessam procurando por algum padrão no endereço e utilizar uma expressão regular para realizar o filtro.

Abaixo, vamos filtrar apenas links para outros artigos da wiki, ignorando âncoras, links para arquivos, etc. Faremos isso nos aproveitando de conhecimento de como um verbete na wiki é organizado. Todos links de artigos estarão sempre dentro da tag **div** que contém um atributo de **id** com valor **'bodyContent'**. Além disso, todo link de verbete necessariamente começa com o endereço "/wiki/" e não possui ":" no endereço.

Podemos generalizar este código em forma de uma função **getLinks()**. Isto possibilitará que busquemos os links de qualquer verbete da wiki.

A função **getLinks()** anterior funciona se quisermos encontrar todos os links de uma única página, porém, se quisermos fazer um crawler efetivo, precisamos procurar por páginas linkadas dentro de outras páginas de forma recursiva. Podemos fazer isso chamando a nosa própria função de procurar link de forma recursiva.

In [8]:
import re

webpages = set()
countriesLink = set()

def getLinks(pageUrl):
    global webpages, countriesLink
    html = urlopen('http://127.0.0.1:8000{}'.format(pageUrl))
    bs = BeautifulSoup(html, 'html.parser')
    for link in bs.find('section',  {'id':'main'}).find_all('a', href=re.compile('^(/places/)((?!:).)*$')):
        if link.attrs['href'] not in webpages:
            #Encontramos um link para uma página nova
            newPage = link.attrs['href']
            if re.match('^(\/places\/default\/view\/)\S*-\d*$', newPage):
                if newPage not in countriesLink:
                    countriesLink.add(newPage)
            if re.match('^(\/places\/default\/index\/)\d*$', newPage):
                if newPage not in webpages:
                    webpages.add(newPage)
                    getLinks(newPage)



Experimente executar o comando a seguir e veja o que acontece. 

In [9]:
getLinks('/places/default/index')
print(sorted(webpages))
print(len(webpages))
print(sorted(countriesLink))
print(len(countriesLink))

['/places/default/index/0', '/places/default/index/1', '/places/default/index/10', '/places/default/index/11', '/places/default/index/12', '/places/default/index/13', '/places/default/index/14', '/places/default/index/15', '/places/default/index/16', '/places/default/index/17', '/places/default/index/18', '/places/default/index/19', '/places/default/index/2', '/places/default/index/20', '/places/default/index/21', '/places/default/index/22', '/places/default/index/23', '/places/default/index/24', '/places/default/index/25', '/places/default/index/3', '/places/default/index/4', '/places/default/index/5', '/places/default/index/6', '/places/default/index/7', '/places/default/index/8', '/places/default/index/9']
26
['/places/default/view/Afghanistan-1', '/places/default/view/Aland-Islands-2', '/places/default/view/Albania-3', '/places/default/view/Algeria-4', '/places/default/view/American-Samoa-5', '/places/default/view/Andorra-6', '/places/default/view/Angola-7', '/places/default/view/A

pegando a lista de paises e coletando as informacoes de cada um

In [31]:
import re
from datetime import datetime
from urllib.error import HTTPError
from urllib.error import URLError

countriesEntity = []

for countryLink in countriesLink:
    print(countryLink)
    try:
        # html = urlopen('http://127.0.0.1:8000{}'.format('/places/default/view/Afghanistan-1'))
        html = urlopen('http://127.0.0.1:8000{}'.format(countryLink))
    except HTTPError as e:
        print("The server returned an HTTP error")
    except URLError as e:
        print("The server could not be found!")
    else:
        bs = BeautifulSoup(html, 'html.parser')

        country = {}
        country['name'] = bs.find('tr',  {'id':'places_country__row'}).find('td',  {'class':'w2p_fw'}).text
        country['capital'] = bs.find('tr',  {'id':'places_capital__row'}).find('td',  {'class':'w2p_fw'}).text
        country['area'] = bs.find('tr',  {'id':'places_area__row'}).find('td',  {'class':'w2p_fw'}).text

        neighbors = [] #acessando o nome dos vizinhos a partir das siglas
        for ng in bs.find('tr',  {'id':'places_neighbours__row'}).find('td',  {'class':'w2p_fw'}).find_all('a'):
            if re.match('^(\/places\/default\/iso\/)[A-Z]{2}$', ng.get('href')):
                html = urlopen('http://127.0.0.1:8000{}'.format(ng.get('href')))
                bs = BeautifulSoup(html, 'html.parser')
                neighbors.append(bs.find('tr',  {'id':'places_country__row'}).find('td',  {'class':'w2p_fw'}).text)

        country['neighbors'] = neighbors
        country['timestamp'] = datetime.now().strftime('%d-%m-%Y %H:%M:%S') # Obter a string de data e hora atual
        countriesEntity.append(country)

/places/default/view/Argentina-11
/places/default/view/Ireland-106
/places/default/view/Cambodia-39
/places/default/view/Laos-121
/places/default/view/Bolivia-27
/places/default/view/Finland-75
/places/default/view/Poland-178
/places/default/view/Canada-41
/places/default/view/Libya-126
/places/default/view/Burkina-Faso-37
/places/default/view/French-Southern-Territories-79
/places/default/view/Madagascar-132
/places/default/view/Mali-136
/places/default/view/Moldova-145
/places/default/view/Vanuatu-244
/places/default/view/Belgium-22
/places/default/view/Botswana-30
/places/default/view/Israel-108
/places/default/view/American-Samoa-5
/places/default/view/Cocos-Islands-49
/places/default/view/US-Virgin-Islands-235
/places/default/view/Iran-104
/places/default/view/Saint-Barthelemy-187
/places/default/view/Nepal-155
/places/default/view/Ghana-84
/places/default/view/Portugal-179
/places/default/view/Puerto-Rico-180
/places/default/view/Turkey-231
/places/default/view/Aland-Islands-2
/p

jogando num csv

In [32]:
import csv

nome_arquivo = "/Users/vitordelela/Documents/PUCRS/7 SEM/ColePrepAnDados/paises.csv"

# gravando os resultados no arquivo CSV.
with open(nome_arquivo, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Nome', 'Capital', 'Area', 'Vizinhos', 'Obtido em'])  # Escreve o cabeçalho do arquivo CSV

    for country in countriesEntity:
        writer.writerow([country['name'], country['capital'], country['area'], country['neighbors'], country['timestamp']])  # Escreve o número da jogada e o resultado no arquivo CSV